In [1]:
liked_books = ["108986",
"81022",
"70355",
"14560521",
"6352404",
"7809996",
"49869",
"49839",
"32737635",
"9692921",
"318862",
"22017381",
"25173758",
"6601394",
"6601396",
"8804484",
"23642748",
"20497356",
]

In [2]:
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [3]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

In [4]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [5]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [6]:
top_recs = recs["book_id"].value_counts().head(10)

In [115]:
top_recs = top_recs.index.values

In [116]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [117]:
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
103618,9712,Love in the Time of Cholera,287272,https://www.goodreads.com/book/show/9712.Love_...,https://images.gr-assets.com/books/1327124987m...,love in the time of cholera
171058,8127,"Anne of Green Gables (Anne of Green Gables, #1)",513174,https://www.goodreads.com/book/show/8127.Anne_...,https://s.gr-assets.com/assets/nophoto/book/11...,anne of green gables anne of green gables 1
175188,29437949,Behind Closed Doors,72344,https://www.goodreads.com/book/show/29437949-b...,https://images.gr-assets.com/books/1458854462m...,behind closed doors
333271,1431,Veronika Decides to Die,114141,https://www.goodreads.com/book/show/1431.Veron...,https://s.gr-assets.com/assets/nophoto/book/11...,veronika decides to die
601765,11899,The Hours,99078,https://www.goodreads.com/book/show/11899.The_...,https://images.gr-assets.com/books/1479663379m...,the hours
792599,15790837,Let's Explore Diabetes with Owls,83765,https://www.goodreads.com/book/show/15790837-l...,https://images.gr-assets.com/books/1359704028m...,lets explore diabetes with owls
1046029,1078,"The Good Earth (House of Earth, #1)",184111,https://www.goodreads.com/book/show/1078.The_G...,https://images.gr-assets.com/books/1312497600m...,the good earth house of earth 1
1055718,1934,"Little Women (Little Women, #1)",1276989,https://www.goodreads.com/book/show/1934.Littl...,https://s.gr-assets.com/assets/nophoto/book/11...,little women little women 1
1202033,2526,Blindness,100752,https://www.goodreads.com/book/show/2526.Blind...,https://images.gr-assets.com/books/1327866409m...,blindness
1223254,968,"The Da Vinci Code (Robert Langdon, #2)",1465770,https://www.goodreads.com/book/show/968.The_Da...,https://images.gr-assets.com/books/1303252999m...,the da vinci code robert langdon 2


In [118]:
all_recs = recs["book_id"].value_counts()

In [119]:
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]

In [120]:
all_recs.head(5)

,book_id,book_count
0,968,3
1,2526,2
2,1431,2
3,8127,2
4,15790837,2


In [121]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [122]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [123]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
281,18511803,1,The Transcriptionist,16,https://www.goodreads.com/book/show/18511803-t...,https://images.gr-assets.com/books/1393791449m...,the transcriptionist,0.062500
566,25467174,1,Housekeeping,16,https://www.goodreads.com/book/show/25467174-h...,https://images.gr-assets.com/books/1430560693m...,housekeeping,0.062500
639,30117415,1,سقوط فرشتگان,16,https://www.goodreads.com/book/show/30117415,https://images.gr-assets.com/books/1462469423m...,,0.062500
644,30900033,1,لی لا لی لا,16,https://www.goodreads.com/book/show/30900033,https://images.gr-assets.com/books/1467812912m...,,0.062500
824,13558998,1,أمريكا: هل هي شعب الله المختار,16,https://www.goodreads.com/book/show/13558998,https://images.gr-assets.com/books/1333126657m...,,0.062500
739,3120436,1,جنگل نروژی,16,https://www.goodreads.com/book/show/3120436,https://images.gr-assets.com/books/1486530672m...,,0.062500
688,17822916,1,من تا صبح بیدارم,18,https://www.goodreads.com/book/show/17822916,https://images.gr-assets.com/books/1366102832m...,,0.055556
970,34051963,1,Here We Lie,18,https://www.goodreads.com/book/show/34051963-h...,https://images.gr-assets.com/books/1499097971m...,here we lie,0.055556
278,35965714,1,The Lying Game,18,https://www.goodreads.com/book/show/35965714-t...,https://images.gr-assets.com/books/1502227209m...,the lying game,0.055556
707,6505230,1,دیوانه‌ای در شهر,18,https://www.goodreads.com/book/show/6505230,https://images.gr-assets.com/books/1369811657m...,,0.055556


In [124]:
all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score


In [125]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [126]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
